# Model creation

In [ ]:
import pandas as pd
import numpy as np
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!gdown https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM

Access denied with the following error:

 	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM 



In [ ]:
!gzip -d GoogleNews-vectors-negative300.bin.gz

gzip: GoogleNews-vectors-negative300.bin.gz: No such file or directory


In [ ]:
from gensim.models import KeyedVectors
from gensim.downloader import load
# model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
model = load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


# Data Preprocessing

In [ ]:
train = pd.read_csv('NLP_ass_train.tsv', sep='\t')
val = pd.read_csv('NLP_ass_valid.tsv', sep='\t')
test = pd.read_csv('NLP_ass_test.tsv', sep='\t')

train.columns = ['text', 'label']
val.columns = ['text', 'label']
test.columns = ['text', 'label']

In [ ]:
train

,text,label
0,nawt yall niggers ignoring me,normal
1,<user> i am bit confused coz chinese ppl can n...,hatespeech
2,and this is why i end up with nigger trainee d...,hatespeech
3,nogs jews and dykes how enriching,offensive
4,guilty until proven innocent unless you are a ...,hatespeech
...,...,...
15377,this vile racist ghetto ass dumber than a reta...,offensive
15378,when i first got on here and said i hate trump...,offensive
15379,was macht der moslem wenn der zion gegen seine...,normal
15380,it is awful look at world demographics asians ...,hatespeech


In [ ]:
test

,text,label
0,laura loomer raped me while screaming at me in...,hatespeech
1,<user> what did the old lady do was she a nazi...,normal
2,as much as i appreciate a format of open discu...,normal
3,sex be so good a bitch be slow stroking and cr...,offensive
4,<user> <user> <user> with a room full of white...,normal
...,...,...
1918,it always women trying this shit like dingbat ...,offensive
1919,because women would never lie about being sexu...,offensive
1920,how is all that awesome muslim diversity going...,offensive
1921,well my dear lgbtq brothers and sisters i do n...,hatespeech


In [ ]:
label2id = {
    'normal': 0,
    'hatespeech': 1,
    'offensive': 2
}

train_text, train_labels = list(train['text']), np.array(list(train['label'].apply(lambda x: label2id[x])))
val_text, val_labels = list(val['text']), np.array(list(val['label'].apply(lambda x: label2id[x])))
test_text, test_labels = list(test['text']), np.array(list(test['label'].apply(lambda x: label2id[x])))

In [ ]:
import re

def preprocess(text):
  clean_text = re.sub(r'[^a-zA-Z0-9\s\!\.\?,]', '', text)  # remove unnecessary characters
  tokens = clean_text.split()
  stop_words = set(stopwords.words('english'))
  tokens = [token for token in tokens if token.lower() not in stop_words]
  return tokens

# Function to convert text to word vectors
def process_data(text, labels, model, vector_size):
    features = []
    new_labels = []
    for sentence, label in zip(text, labels):
        filtered_tokens = preprocess(sentence)
        sentence_vector = np.mean([model[word] for word in filtered_tokens if word in model], axis=0)
        if type(sentence_vector) == np.ndarray and len(sentence_vector) == vector_size:
          features.append(sentence_vector)
          new_labels.append(label)
    return np.vstack(features), np.array(new_labels)

vector_size = model.vector_size
train_features, train_labels = process_data(train_text, train_labels, model, vector_size)
val_features, val_labels = process_data(val_text, val_labels, model, vector_size)
test_features, test_labels = process_data(test_text, test_labels, model, vector_size)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
import tensorflow as tf
from keras.utils import to_categorical

train_features = tf.convert_to_tensor(train_features, dtype=tf.float32)
val_features = tf.convert_to_tensor(val_features, dtype=tf.float32)
test_features = tf.convert_to_tensor(test_features, dtype=tf.float32)

train_labels = tf.convert_to_tensor(to_categorical(train_labels, num_classes=3), dtype=tf.float32)
val_labels = tf.convert_to_tensor(to_categorical(val_labels, num_classes=3), dtype=tf.float32)
test_labels = tf.convert_to_tensor(to_categorical(test_labels, num_classes=3), dtype=tf.float32)

In [ ]:
test_labels.shape

TensorShape([1922, 3])

In [ ]:
assert train_features.shape[0] == train_labels.shape[0]
assert val_features.shape[0] == val_labels.shape[0]
assert test_features.shape[0] == test_labels.shape[0]

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

ff_model = Sequential()
ff_model.add(Dense(64, activation='relu', input_dim=vector_size))
ff_model.add(Dense(128, activation='relu'))
ff_model.add(Dense(3, activation='softmax'))

early_stopper = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
ff_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
ff_model.fit(train_features, train_labels, epochs=15, batch_size=32, validation_data=(val_features, val_labels), callbacks=[early_stopper], verbose=1)
ff_model.save('best_model_ckpt.h5')  # Save the best model

Epoch 1/15
481/481 [==============================] - 2s 3ms/step - loss: 0.9105 - accuracy: 0.5790 - val_loss: 0.8787 - val_accuracy: 0.6047
Epoch 2/15
481/481 [==============================] - 2s 3ms/step - loss: 0.8421 - accuracy: 0.6245 - val_loss: 0.8670 - val_accuracy: 0.6203
Epoch 3/15
481/481 [==============================] - 2s 3ms/step - loss: 0.8161 - accuracy: 0.6377 - val_loss: 0.8617 - val_accuracy: 0.6172
Epoch 4/15
481/481 [==============================] - 1s 2ms/step - loss: 0.7975 - accuracy: 0.6471 - val_loss: 0.8524 - val_accuracy: 0.6219
Epoch 5/15
481/481 [==============================] - 1s 2ms/step - loss: 0.7759 - accuracy: 0.6575 - val_loss: 0.8516 - val_accuracy: 0.6198
Epoch 6/15
481/481 [==============================] - 1s 2ms/step - loss: 0.7561 - accuracy: 0.6632 - val_loss: 0.8538 - val_accuracy: 0.6266
Epoch 7/15
481/481 [==============================] - 1s 3ms/step - loss: 0.7373 - accuracy: 0.6790 - val_loss: 0.8688 - val_accuracy: 0.6146
Epoch 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from sklearn.metrics import f1_score
from keras.models import load_model

finetuned_model = load_model('best_model_ckpt.h5')
logits = finetuned_model(test_features)
predictions = tf.argmax(logits, axis=1)
print(predictions.shape)

test_loss, test_accuracy = finetuned_model.evaluate(test_features, test_labels, verbose=1)
test_labs = tf.argmax(test_labels, axis=1)
macro_f1 = f1_score(test_labs, predictions, average='macro')
print(f'Test Accuracy: {test_accuracy:.4f}, Macro-F1: {macro_f1:.4f}')

(1922,)
61/61 [==============================] - 0s 2ms/step - loss: 0.8399 - accuracy: 0.6249
Test Accuracy: 0.6249, Macro-F1: 0.5931


In [ ]:
def find_common_strings(sentences1, sentences2):
  sent1 = set(sentences1)
  sent2 = set(sentences2)

  return len(sent1.intersection(sent2))

In [ ]:
print("Number of common sentences (train, test): ", find_common_strings(train_text, test_text))
print("Number of common sentences (val, test): ", find_common_strings(val_text, test_text))

Number of common sentences (train, test):  5
Number of common sentences (val, test):  1
